# Setup for PyTorch Mask-RCNN

## Installs

System (Linux and NVIDIA GPU required with CUDA/cuDNN):
* CUDA 9.0
* NVIDIA Tesla K80
* Ubuntu 16.04


The Python used (tested with Anaconda 3.5, 3.6, and 3.7):
* DSVM:  `/anaconda/envs/py35/bin/python` or `/anaconda/envs/pytorch041/bin/python` if available
* macOS:  `/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/`


In [3]:
! nvidia-smi

Sun May 12 03:17:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00004BBC:00:00.0 Off |                    0 |
| N/A   34C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
# What Python
import sys
sys.executable

'/anaconda/envs/pytorch041/bin/python'

In [ ]:
# Install libs from requirements.txt library
! sudo LC_ALL=C {sys.executable} -m pip install -r requirements.txt

In [5]:
# What PyTorch
import torch
torch.__version__

'0.4.1.post2'

## Building the extensions

Could also do:

```
! export PATH=/usr/local/cuda/bin:$PATH &&\
    export USE_OPENCV=1 &&\
    export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH &&\
    export LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/usr/local/cuda/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda/ &&\
    export CUDNN_LIB_DIR=/usr/local/cuda/lib64 &&\
    export CUDA_HOST_COMPILER=cc &&\
    export USE_CUDA=1 &&\
    export USE_NNPACK=1 &&\
    cd nms/src/cuda/ && nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    TORCH_CUDA_ARCH_LIST="3.5 5.2 6.0 6.1+PTX" TORCH_NVCC_FLAGS="-Xfatbin -compress-all" &&\
    CC=cc CXX=cc++ python3 build.py
```

**NOTE**: Update the path to Python executable below before running the following bash script

In [16]:
%%bash

export PATH=/anaconda/envs/pytorch041/bin/:$PATH

CUDA_ARCH="-gencode arch=compute_30,code=sm_30 \
           -gencode arch=compute_35,code=sm_35 \
           -gencode arch=compute_50,code=sm_50 \
           -gencode arch=compute_52,code=sm_52 \
           -gencode arch=compute_60,code=sm_60 \
           -gencode arch=compute_61,code=sm_61 \
	   -gencode arch=compute_70,code=sm_70 "


# Build RoIAlign
cd lib/roi_align/src/cuda
echo 'Compiling crop_and_resize kernels by nvcc...'
/usr/local/cuda/bin/nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC $CUDA_ARCH
cd ../../
python build.py
cd ../..

# Build NMS
cd lib/nms/src/cuda
echo "Compiling nms kernels by nvcc..."
/usr/local/cuda/bin/nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC $CUDA_ARCH
cd ../../
python build.py
cd ../..

Compiling crop_and_resize kernels by nvcc...
Including CUDA code.
/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/roi_align
generating /tmp/tmp7w45x_xn/_crop_and_resize.c
setting the current directory to '/tmp/tmp7w45x_xn'
running build_ext
building '_crop_and_resize' extension
creating data
creating data/home
creating data/home/wonderwoman
creating data/home/wonderwoman/notebooks
creating data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples
creating data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib
creating data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/roi_align
creating data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/roi_align/src
gcc -pthread -B /data/anaconda/envs/pytorch041/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DWITH_CUDA -I/anaconda/envs/pytorch041/lib/python3.7/site-packages/torch/utils/ffi/../../lib/include -I/anaconda/envs/pytorch041/lib/python3.7/sit

/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/roi_align/src/crop_and_resize.c: In function ‘CropAndResizePerBox’:
/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/roi_align/src/crop_and_resize.c:30:0: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
     #pragma omp parallel for
 ^
/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/nms/src/nms_cuda.c: In function ‘gpu_nms’:
/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/nms/src/nms_cuda.c:29:35: warning: initialization from incompatible pointer type [-Wincompatible-pointer-types]
   unsigned long long* mask_flat = THCudaLongTensor_data(state, mask);
                                   ^
/data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/lib/nms/src/nms_cuda.c:37:40: warning: initialization from incompatible pointer type [-Wincompatible-pointer-types]
   unsigned long long * mask_cpu_flat = THLongTensor_data(mask_cpu);
                                        ^
/

## `pycocotools` and `coco` libraries

**NOTE**: Update the path to Python executable below before running the following bash script

In [20]:
%%bash

export PATH=/anaconda/envs/pytorch041/bin/:$PATH

rm -fr coco
rm -fr cocoapi
rm -fr pycocotools

export CC=cc
export CXX=cc++

git clone https://github.com/waleedka/coco.git
# git clone https://github.com/cocodataset/cocoapi.git &&\
cd coco/PythonAPI/ && python setup.py build_ext --inplace
cd ../..
ln -s coco/PythonAPI/pycocotools/ pycocotools

Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
running build_ext
building 'pycocotools._mask' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
creating build/common
cc -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/anaconda/envs/pytorch041/lib/python3.7/site-packages/numpy/core/include -I../common -I/anaconda/envs/pytorch041/include/python3.7m -c pycocotools/_mask.c -o build/temp.linux-x86_64-3.7/pycocotools/_mask.o -Wno-cpp -Wno-unused-function -std=c99
cc -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/anaconda/envs/pytorch041/lib/python3.7/site-packages/numpy/core/include -I../common -I/anaconda/envs/pytorch041/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
gcc -pthread -shared -B /data/anaconda/envs/pytorch041/compiler_compat -L/data/anacond

Cloning into 'coco'...
/anaconda/envs/pytorch041/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /data/home/wonderwoman/notebooks/pytorch-mask-rcnn-samples/coco/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [21]:
! ls

build_pytorch1.0_macosx_10.13.sh  lib
coco				  LICENSE
coco.py				  model.py
config.py			  __pycache__
data				  pycocotools
Demo.ipynb			  README.md
demo.py				  requirements.txt
fish_maskrcnn.zip		  Setup.ipynb
fish_pytorch_style.py		  Train.ipynb
images				  utils.py
InstallPyTorchSourceCPU.ipynb	  visualize.py
InstallPyTorchSourceCUDA.ipynb


Note, here, for macOS, using Xcode 9.4.1 command line tools

In [ ]:
# # MacOS, see https://pytorch.org/ for other distributions
# ! pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

More relevant resolved GitHub issues:
* https://github.com/uber/horovod/issues/274#issuecomment-390813445
* https://github.com/pytorch/pytorch/issues/10234